# Bayesian Neural Networks Notes

## Deep Neural Networks Uncertainty

- The main benefit of using a Bayesian framework is that allows to propagate uncertainty to improve prediction and model reasoning. In this sense, is important that each component of a system computes an accurate estimate of uncertainty to later reflect the true uncertaint in the system.

- Metrics must capture must capture the quantitative performance of the uncertainty the model is trying to measure.

- Typical metrics should include uncertainty calibration plots and precision recall statistics of regression and classification algorithms.

- Task-specific metrics include inspecting __test-log-predictive-probabilities a measure of how “surprised”__ the system output is on a test-set of known outcomes

- It is most important to accurately __quantify the uncertainty of intermediate representations__, such as those from perception and scene prediction components.

- The most important aspect is to correctly propagate uncertainty to the following layers so that we can make informed decisions. The final control output will be a hard decision. Therefore uncertainty metrics should focus on intermediate components and tasks such as perception, sensor fusion and scene prediction


__Notes Taken from [1]__

### Bayesian Uncertainty Estimation

- Uncertainty is Used to track the dependability of the model.
- In classificaiton problems, softmax probabilities are not enough to indicate whether the model is confident in its prediction.
- A standard model would pass a point estimate through the softmax layer rather than the whole predictive distribution. This lead to high probabilities or high confidence on points far from data.
    - __An example of Out-of-Distribution sample:__ In self-driving, if the training set of a model consisted of only images from highways and the model was then given an image of a dirt track, the model would return a steering angle but we would ideally require the model to have high uncertainty
    - Noisy data $\rightarrow$ Aleatoric uncertainty
    - Model parameters uncertainty/variability $\rightarrow$ Epistemic uncertainty
    
-  Means to estimate epistemic uncertainty is to perform __Bayesian Inference__.


__Notes taken from [2]__

### Epistemic Uncertainty Estimation with Bayesian Inference

The aim is to use the posterior distribution $p(\theta \mid D)$ which is obatained from the data likelihood and the prior $p(\theta)$ by applying Bayes theorem. The uncertainty in the model parameters is then marginalized out to obtain the posterior distribution:


<center>$p(y^{*} \mid x^{*}, D) = \int p(y^{*} \mid x^{*}, \theta) p(\theta \mid D) d\theta$</center>

The general intractable integral from above is approximated using $M$ Monte Carlo samples $\theta^{i}$, ideally from the posterior. However, in practice is virtually impossible for DNNs to obtain samples from the true posterior $p(\theta \mid D)$. This requires approximation methods $q(\theta) \approx p(\theta \mid D)$ to be used [4].

<center>$p(\theta \mid D) = \frac{p(D \mid \theta) p(\theta)}{p(D)}$</center>

The probability distribution of interest is the distribution p of the parameters θ after the data D is seen. p(D|θ) is the likelihood of the occurrence of data D given a model with parameters θ, p(θ) the prior, and p(D) the data distribution. Incorporating a prior belief in investigating a posterior state is a central characteristic of Bayesian reasoning. This prior is hence often referred to as domain knowledge [5].

Learning for the BNN, amounts to computing the posterior distribution over the weights, $p(w \mid D)$ through Bayes Rule. Because of the non-linearity introduced by the Neural Network architecture, the computation of the posterior cannot be done analytically. Hence varios approximation smethods have been studied to perform inference with BNNs in practice [3]:


1. __Markov Chain Monte Carlo Methods__
    - __Hamiltonian Monte Carlo (HMC):__
    
    Proceeds by defining a Markov chain whose invariant distribution is $p(w \mid D)$ or $p(\theta \mid D)$. Relies on Hamiltonian dynamics to speed up the exploration of the space. HMC does not make any assumptions on the form of the posterior distribution, and is asymptotically correct. The result of HMC is a set of samples $w_{i}$ or $\theta_{i}$ that approximate $p(w \mid D)$ or $p(\theta \mid D)$ [3].
    

2. __Variation Inference Methods__
    Proceeds by finding a Gaussian approximate distribution $q(w) \approx p(w \mid D)$ in a trade-off between a pproximation accuracy and scalability. The core idea is that $q(w)$ depends on some hyper-parameters that are then iteratively optimized by minimizing a divergenec measeure between $q(w)$ and $p(w \mid D)$. Samples can then be efficientyl extracted from $q(w)$ [3]
    
    - __Monte Carlo Dropout (MCD):__
    A particularly simple and scalable variant. The method entails using dropout also at __test time__, which can be interpreted as performing Variational Inference (VI) with a Bernoulli variational distribution. The approximate predictive posterior is obtained by performing $M$ stochastic forward passes on the same input [4].
    
    <center>$p(y^{*} \mid x^{*}, D):= \frac{1}{M} \sum_{i=1}^{M}p(y^{*} \mid x^{*}, \theta^{(i)}), \space \space \theta^{(i)} \sim p(\theta \mid D)$</center>
    

3. __Ensembling (considerend also non-bayesian)__
The parametric model $p(y \mid x, \theta)$ of the conditional distribution was created using a DNN $f_{\theta}$, and learned multiple point estimates $\{\hat{\theta}^{(m)}\}_{m=1}^{M}$, by repeatedly minimizing the MLE objective $log\space p(Y \mid X, \theta)$ with random initialization. They then averaged over the corresponding parametric models to obtain the predictive distribution:

<center>$p(y^{*} \mid x^{*}, D):= \frac{1}{M} \sum_{m=1}^{M}p(y^{*} \mid x^{*}, \theta^{(m)})$</center>

This approach is considered as a non-Bayesian alternative to predictive uncertainty estimation. However, since  $\{\hat{\theta}^{(m)}\}_{m=1}^{M}$, always can be seen as samples from some distribution $\hat{q}(\theta)$, we note that the previous equation is virtually identical to the equation described in MCD section. Ensembling can also be viewed as approximate Bayesian inference, where the level of approximation is determined by how well the implicit sampling distribution $\hat{q}(\theta)$ approximates the posterior $p(\theta \mid D)$ [4].


## References

1. _Concrete Problems for Autonomous Vehicle Safety: Advantages of Bayesian Deep Learning_. R McAllister, Y. Gal, A. Kendall, et al. IJCAI 2017

2. _Evaluating Quantification in End-to-End Autonomous Driving Control_. R Michelmore, M Kwiatkowska, Y Gal. 16/11/2018

3. _Uncertainty Quantification with Statistical Guaranteees in End-to-End Autonomous Driving Control_. R Michelmore, M Wicker, L Laurenti, L Cardelli, Y Gal, M Kwiatkowska. 21/09/2019

4. _Evaluating Scalable Bayesian Deep Learning Methods for Robust Computer Vision_. F K Gustafsson, M Danelljan,  et al. 04/06/2019

5. [_Probabilistic Deep Learning: Bayes by Backprop_. Felix Laumann.](https://medium.com/neuralspace/probabilistic-deep-learning-bayes-by-backprop-c4a3de0d9743)
